# AF Classification Preprocessing with Spectrogram Approach
The following notebook describes the code used to process ECG signals in order to be ready for training and evaluation from the multi - classificator built for this course project.

## Libraries and general variables.

In [54]:
import numpy as np
import pandas as pd
import sys
import wfdb
from wfdb import processing 
from scipy.signal import medfilt
import scipy
import keras
import pickle
import os.path
import QRS_util
from tqdm import tqdm
from random import seed
from sklearn.model_selection import train_test_split

folderPath = "./training2017/"
recordsPath = folderPath + 'REFERENCE.csv'

## Loading signals
The preprocessing part starts by loading signals from Physionet Challenge and storing them, together with their labels, into data structures.

In [27]:
def LoadSignalsAndLabelsFromFile(folderPath):

    signals = []
    recordsFilePath = folderPath + 'REFERENCE.csv'
    recordsAndLabels = pd.read_csv(recordsFilePath, header=None, names=['filename', 'label'])

    if os.path.isfile("./RawSignals.pk1") == False:

        print('Getting raw signals ...')
        for recordName in recordsAndLabels['filename']:     
            recordName = folderPath + recordName
            record = wfdb.rdrecord(recordName)
            digitalSignal = record.adc()[:,0]
            signals.append(digitalSignal)

        print('Done.')
        print('Saving signals to file ...')
        with open ('./RawSignals.pk1', 'wb') as f:
            pickle.dump(signals, f)
    else:
        print('Loading raw signals ...')

        with open('./RawSignals.pk1', 'rb') as fp:
            signals = pickle.load(fp)
            print('Done.') 

    signals = np.array(signals)
    y = recordsAndLabels['label']

    return signals, y

## Splitting dataset into training and test set
Now our dataset is created, we are going to divide it into a training and test set. We splitted it with an 80 / 20 approach and with a stratify approach, which made us able to mantain this percentage split for each class of the dataset. 

In [28]:
def TrainTestSplit(signals, labels):

    if len(signals) and len(labels)> 0:
        df = {

            'signal' : signals,
            'label' : labels
        }

        df = pd.DataFrame(df, columns = ['signal', 'label'])

        # Keep 20% of the data out for validation
        train_reference_df, val_reference_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=123)

        # 'N' = 0
        # 'A' = 1
        # 'O' = 2
        # '~' = 3

        return train_reference_df, val_reference_df

## Baseline wander removal using Butterworth filter
In order to remove baseline wander from ECGs, we passed them through a Butterworth filter.

In [29]:
def BaselineWanderFilter(signals):

    print('Filtering ...')
    # Sampling frequency
    fs = 300

    for i, signal in enumerate(signals):

        # Baseline estimation
        win_size = int(np.round(0.2 * fs)) + 1
        baseline = medfilt(signal, win_size)
        win_size = int(np.round(0.6 * fs)) + 1
        baseline = medfilt(baseline, win_size)

        # Removing baseline
        filt_data = signal - baseline
        signals[i] = filt_data
    
    print('Storing filtered signals..')
    with open ('./FilteredSignals.pk1', 'wb') as f:
        pickle.dump((signals), f)
    print('Done.')

    return signals

## Performing data augmentation on training set
In order to increment the number of the observations for the test set, we doubled up the number of signals for AFib and Noise class.

In [30]:
def CreateNoiseVector():
    
    noise = np.random.normal(0, 2000, size = (9000,))
    noise = noise.astype(np.int64)
    return noise

def TrainingTestAugumentationSpectrogram(trainingSet):

    y = trainingSet['label']
    signals = trainingSet['signal']
    signals = signals.to_numpy()

    print('Class distribution before doubling up ...')
    print('Normal: ', len(np.where(y == 'N')[0]))
    print('AF: ', len(np.where(y == 'A')[0]))
    print('Other: ', len(np.where(y == 'O')[0]))
    print('Noisy: ', len(np.where(y == '~')[0]))

    # Double up AF signals

    aFibMask = np.where(y == 'A')
    aFibSignals = np.array(signals[aFibMask])

    signals = np.hstack((signals, aFibSignals))
    newAfibLabels = ['A']*len(aFibSignals)
    y = np.append(y, newAfibLabels)

    print('New AF observations number: ', len(np.where(y == 'A')[0]))

    # Duplicating noisy signals to add

    noiseMask = np.where(y == '~')
    noiseSignals = np.array(signals[noiseMask])

    for i, signal in enumerate(noiseSignals):
        noiseSignals[i] = CreateNoiseVector()

    signals = np.hstack((signals, noiseSignals))
    #   signals = np.hstack((signals, noiseSignals))
    newNoiseLabels = ['~']*len(noiseSignals)
    y = np.append(y, newNoiseLabels)

    print('New Noise observations number: ', len(np.where(y == '~')[0]))
       
    print('Done.')

    dataset = {

        'signal' : signals,
        'label' : y
    }

    dataset = pd.DataFrame(dataset, columns = ['signal', 'label'])

    print('Storing signals and labels to file..')
    with open ('./TrainingTestAugmented.pk1', 'wb') as f:
        pickle.dump((dataset), f)
        
    return dataset

## Cropping signals into a fixed length
In order to create a machine-friendly format, all the ECGs were cropped up into a fixed length of 9000. The approach used for cropping depends on the nature of the dataset (training or test).

In [31]:
def RandomCrop(df, target_size=9000, center_crop=True):
    
    signals = df['signal']
    labels = df['label']
    newSignals = []
    print('Cropping data ...')

    for i, data in enumerate(signals):

        N = data.shape[0]
        
        # Return data if correct size
        if N == target_size:
            newSignals.append(data)
            continue
        
        # If data is too small, then pad with zeros
        if N < target_size:
            tot_pads = target_size - N
            left_pads = int(np.ceil(tot_pads / 2))
            right_pads = int(np.floor(tot_pads / 2))
            newSignal = np.pad(data, [left_pads, right_pads], mode='constant')
            newSignals.append(newSignal)
            continue

        # Random Crop (always centered if center_crop=True)
        if center_crop:
            from_ = int((N / 2) - (target_size / 2))
        else:
            from_ = np.random.randint(0, np.floor(N - target_size))
        to_ = from_ + target_size
        newSignal = data[from_:to_]
        newSignals.append(newSignal)
    
    dataset = {

        'signal' : newSignals,
        'label' : labels
    }

    dataset = pd.DataFrame(dataset, columns = ['signal', 'label'])

    print('Done.')
    return dataset

## Performing FFT and its logarithm
To use an ECG signal as an image, we had to use Fast Fourier Transform (FFT) in order to obtain a 2D spectrogram. Logarithm is then used to normalize data.

In [32]:
def GenerateSpectrogramFromSignal(signal):
    _, _, sg = scipy.signal.spectrogram(signal, fs=300, window=('tukey', 0.25), 
            nperseg=64, noverlap=0.5, return_onesided=True)

    return sg.T

def FFT(dataset):

    signals = dataset['signal']
    labels = dataset['label']

    print('Computing FFTs ...')
    logSpectrograms = []

    for s in signals:

        logSp = np.log(GenerateSpectrogramFromSignal(s) + 1)
        if logSp.shape[0] != 140:
            print('Shape diverso da 140')
            continue
        logSpectrograms.append(logSp)

    logSpectrograms = np.array(logSpectrograms)
    means = logSpectrograms.mean(axis=(1,2))
    stds = logSpectrograms.std(axis=(1,2))
    logSpectrograms = np.array([(log - mean) / std for log, mean, std in zip(logSpectrograms, means, stds)])
    logSignals = logSpectrograms[..., np.newaxis]
    
    print('Storing log signals to file..')
    with open ('./LogSignals.pk1', 'wb') as f:
        pickle.dump(logSignals, f)

    print('Done.')

    return logSignals, labels

## Normalizing data
The last preprocessing step consists on normalizing data by taking the signals belonging to the 5% and 99% of the total. This becomes our normalization factor, and each signal value is divided by that.

In [33]:
def NormalizeData(signals, labels):

    print('Normalizing data ...')
    for i, signal in enumerate(signals):

        # Amplitude estimate
        norm_factor = np.percentile(signal, 99) - np.percentile(signal, 5)
        signals[i] = signal / norm_factor

    print('Done.')
    return signals, labels

## Preprocessing wrapping up method

In [34]:
def PreprocessingForSpectrogramApproach():
    signals, labels = LoadSignalsAndLabelsFromFile(folderPath)  

    if os.path.isfile('./FilteredSignals.pk1'):
        print('Loading previously filtered signals ...')
        with open('./FilteredSignals.pk1', 'rb') as fp:
            signals = pickle.load(fp)
    else:
        signals = BaselineWanderFilter(signals)

    trainingSet, testSet = TrainTestSplit(signals, labels)
    trainingSet = TrainingTestAugumentationSpectrogram(trainingSet)
    trainingSet = RandomCrop(trainingSet, center_crop = False)
    trainSignals, labels = FFT(trainingSet)
    trainSignals, traininglabels = NormalizeData(trainSignals, labels) 
    
    testSet = RandomCrop(testSet, center_crop = True)
    testSignals, labels = FFT(testSet)
    testSignals, testLabels = NormalizeData(testSignals, labels) 

    print('Storing training set to file..')
    with open('./TrainingSetFFT.pk1', 'wb') as f:
        print('Len  train:' + str(len(trainSignals)))
        pickle.dump((trainSignals, traininglabels), f)
            
    print('Storing test set to file..')
    with open ('./TestSetFFT.pk1', 'wb') as f:
        print('Len  test:' + str(len(testSignals)))
        pickle.dump((testSignals, testLabels), f)

## Running it

In [ ]:
PreprocessingForSpectrogramApproach()

Getting raw signals ...
Done.
Saving signals to file ...
Filtering ...
